<a href="https://colab.research.google.com/github/KonradGonrad/PyTorch-deep-learning/blob/main/06_PyTorch_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import torch
  import torchvision
  assert (int(torch.__version__.split('.')[0]) > 1) or \
          (int(torch.__version__.split('.')[0]) == 1 and int(torch.__version__.split('.')[1]) >= 12), 'torch must have newer version than 1.12'
  assert (int(torchvision.__version__.split('.')[1]) >= 13), 'torchvision must be newer version than 0.13'
except:
  print(f'[INFO] torch/torchvision version not as required. Installing newer version')
  !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
  import torch
  import torchvision
  print(f'torch version: {torch.__version__}')
  print(f'torchvision version: {torchvision.__version__}')

In [ ]:
import os
import zipfile
import matplotlib.pyplot as plt
import requests

from pathlib import Path

DATA_PATH = Path('data/')
IMAGE_PATH = DATA_PATH / 'pizza_steak_sushi'

if IMAGE_PATH.is_dir():
  print(f"{IMAGE_PATH} directory already exists")
else:
  print(f"Did not find {IMAGE_PATH} directory, creating one...")
  IMAGE_PATH.mkdir(parents=True, exist_ok=True)

  with open(DATA_PATH / 'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    f.write(request.content)

  with zipfile.ZipFile(DATA_PATH / 'pizza_steak_sushi.zip', 'r') as zipf:
    zipf.extractall(IMAGE_PATH)

  os.remove(DATA_PATH / 'pizza_steak_sushi.zip')


In [ ]:
TRAIN_DIR = IMAGE_PATH / 'train'
TEST_DIR = IMAGE_PATH / 'test'

In [ ]:
Additional_func_path = Path('add/')

if Additional_func_path.is_dir():
  print(f'{Additional_func_path} already exists.')
else:
  print(f"Can't find {Additional_func_path}. Creating one..")
  Additional_func_path.mkdir(parents=True, exist_ok = True)
  with open(Additional_func_path / 'notebook_5.ipynb', 'wb') as f:
    request = requests.get('https://github.com/KonradGonrad/PyTorch-deep-learning/raw/main/Kopia_notatnika_05_pytorch_going_modular_exercise_template.ipynb')

    f.write(request.content)

In [ ]:
!jupyter nbconvert --to script add/notebook_5.ipynb
!mv add/notebook_5.txt add/notebook_5.py


In [ ]:
!pip install nbimporter

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path('add')))

In [ ]:
sys.path

In [ ]:
code_path = 'add/notebook_5.py'

with open(code_path, 'r') as file:
  lines = file.readlines()

  lines_to_keep = [line for i, line in enumerate(lines) if i <= 425]

with open('add/notebook.py', 'w') as file:
  file.writelines(lines_to_keep)

os.remove(code_path)
os.remove('add/notebook_5.ipynb')

In [ ]:
import nbimporter

from notebook import prepare_data